## Aufgaben

In [1]:
import torch
import numpy as np

#### Beispiel 1:

Bestimme den Wert der Funktion Funktion $f(x_1,x_2,x_3) = (x_1^2 + 3x_2) \cdot 2x_3$ am Punkt $x = (2,-1,3)$.
Gehe von $x$ aus um $0.01$ Längeneinheiten in Richtung des negativen Gradienten zum Punkt $x'$ und werte dort die Funktion erneut aus.

<img src="./img/nn_19.png" width="800"/>   

In [2]:
x = torch.tensor([2.0,-1.0,3.0],requires_grad=True)
y = (x[0]*x[0] + 3*x[1])*2*x[2]
print("y =", y.detach().numpy())
y.backward()
grad = x.grad.numpy()
print("Gradient: ",grad)
gradLength = np.linalg.norm(grad)   
print("Länge des Gradienten:", gradLength )
print("Neuer Punkt: ", x.detach().numpy() - 0.01 * grad/gradLength)

y = 6.0
Gradient:  [24. 18.  2.]
Länge des Gradienten: 30.066593
Neuer Punkt:  [ 1.9920177 -1.0059867  2.9993348]


#### Beispiel 2:
Bestimme den Wert der Funktion $f(x_1,x_2,x_3) = (x_1\cdot x_2 + x_1^2) \cdot (x_3 - 5)$ am Punkt $x = (1,-2,2)$.
Gehe von $x$ aus um $0.01$ Längeneinheiten in Richtung des negativen Gradienten zum Punkt $x'$ und werte dort die Funktion erneut aus.
(Mit etwas Übung kann man bei einfachen Funktionen die partiellen Ableitungen direkt hinschreiben). 

<img src="./img/nn_20.png" width="800"/>  

In [3]:
x = torch.tensor([1.0,-2.0,2.0],requires_grad=True)
y = (x[0]*x[1] + x[0]**2)*(x[2]-5)
print("y =", y.detach().numpy())
y.backward()
grad = x.grad.numpy()
print("Gradient: ",grad)
gradLength = np.linalg.norm(grad)   
print("Länge des Gradienten:", gradLength )
print("Neuer Punkt: ", x.detach().numpy() - 0.01 * grad/gradLength)

y = 3.0
Gradient:  [ 0. -3. -1.]
Länge des Gradienten: 3.1622777
Neuer Punkt:  [ 1.        -1.9905132  2.0031624]



#### Beispiel 3: 
Wenn es auf dem Weg zurück zur Variablen mehrere Wege gibt, werden die Werte der Pfade addiert.

Bestimme den Wert der Funktion $f(x_1,x_2,x_3) = (x_1 + 2x_2)(x_2 +4x_3)$ am Punkt $x = (-1,2,3)$. Gehe von $x$ aus um $0.02$ Längeneinheiten in Richtung des negativen Gradienten zum Punkt $x'$ und werte dort die Funktion erneut aus.

<img src="./img/nn_21.png" width="900"/>   


In [4]:
x = torch.tensor([-1,2.0,3.0],requires_grad=True)
y = (x[0]+2*x[1])*(x[1] + 4*x[2])
print("y =", y.detach().numpy())
y.backward()
grad = x.grad.numpy()
print("Gradient: ",grad)
gradLength = np.linalg.norm(grad)   
print("Länge des Gradienten:", gradLength )
print("Neuer Punkt: ", x.detach().numpy() - 0.01 * grad/gradLength)

y = 42.0
Gradient:  [14. 31. 12.]
Länge des Gradienten: 36.069378
Neuer Punkt:  [-1.0038815  1.9914055  2.996673 ]


#### Übung

Berechne für $x = (3,2)$ den Output und die lokalen Gradienten des Neurons <br>
```n = Neuron([-1,2,-4])``` <br>


In [5]:
import torch
import numpy as np
x = torch.tensor([3.0,2.0],requires_grad=False)
n = torch.tensor([-1.0,2.0,-4.0],requires_grad=True)
z = x[0]*n[0] + x[1]*n[1] + n[2]
a = torch.sigmoid(z)
print("a =", a)
a.backward()
grad = n.grad.numpy()
print("Gradient: ",grad)


a = tensor(0.0474, grad_fn=<SigmoidBackward>)
Gradient:  [0.13552998 0.09035332 0.04517666]


#### Beispiel

Für das Neuron  &nbsp; ```n = Neuron([-1,2,-4])```  &nbsp; ist $y = 1$ der erwartete Output bei $x = (-1,2)$.
Berechne die neuen Werte für $w_1, w_2, b$ nach einem Durchgang forward/backward-propagation bei einer learning-rate von 0.2

<img src="./img/nn_23.png" width="801"/>  

In [7]:
import torch
import numpy as np
x = torch.tensor([-1.,2.0],requires_grad=False)
n = torch.tensor([-1., 2., -4.],requires_grad=True)
print("Parameter: ", n.detach().numpy())

for i in range(3):
    z = x[0]*n[0] + x[1]*n[1] + n[2]
    a = torch.sigmoid(z) 
    loss = 0.5*(1 - a)**2

    loss.backward()
    print(n.grad)
    with torch.no_grad():
        n -= 0.2 * n.grad
        
    print("Parameter: ", n.detach().numpy() - 0.2 * grad)
    n.grad.zero_()
    #b = -3.9796, w1 = -1.0204, w2 = 2.0407

Parameter:  [-1.  2. -4.]
tensor([ 0.0529, -0.1058, -0.0529])
Parameter:  [-1.0376815  2.0030801 -3.9984598]
tensor([ 0.0490, -0.0979, -0.0490])
Parameter:  [-1.0474743  2.0226657 -3.988667 ]
tensor([ 0.0455, -0.0910, -0.0455])
Parameter:  [-1.0565753  2.0408678 -3.9795659]


In [10]:
import math
import random

class Neuron:
    def __init__(self,param):
        self.w1, self.w2, self.b = param
    
    def forward(self, x1, x2):
        z = self.w1 * x1 + self.w2 * x2 + self.b
        a = 1/(1+math.exp(-1.0*z))
        
        # wir berechnen die lokalen Gradienten
        self.db = a *(1-a)    
        self.dw1 = x1 * self.db   
        self.dw2 = x2 * self.db
        self.dx1 = self.w1 * self.db
        self.dx2 = self.w2 * self.db
        return a
    
    def backward(self, g):
        
        # wir multiplizieren den lokalen Gradienten mit
        # dem upstream Gradienten g
        self.db *= g
        self.dw1 *= g
        self.dw2 *= g
        self.dx1 *= g
        self.dx2 *= g
        
    def update(self,lr):
        self.w1 = self.w1 - lr * self.dw1
        self.w2 = self.w2 - lr * self.dw2
        self.b = self.b - lr * self.db


n = Neuron([-1,2,-4])
y = 1                  # gewünschte Ausgabe
x1, x2 = -1, 2
lr = 0.2               # learning rate

print('Ausgangslage:')
print('x1 = {:5.4f}, x2 = {:5.4f}'.format(x1, x2)) 
print('b = {:5.4f}, w1 = {:5.4f}, w2 = {:5.4f}'.format(n.b, n.w1, n.w2)) 
print()

for i in range(2):
    print('forward: -----')
    a = n.forward(x1,x2)
    print('a = {:5.4f}, y = {:5.4f}, a-y = {:5.4f}'.format(a,y,a-y))
    print('db = {:5.4f}, dw1 = {:5.4f}, dw2 = {:5.4f}'.format(n.db, n.dw1, n.dw2))  

    print('backward: ----')
    n.backward(a-y)
    n.update(lr)

    print('db = {:5.4f}, dw1 = {:5.4f}, dw2 = {:5.4f}'.format(n.db, n.dw1, n.dw2)) 
    print('b = {:5.4f}, w1 = {:5.4f}, w2 = {:5.4f}'.format(n.b, n.w1, n.w2)) 
    print()




Ausgangslage:
x1 = -1.0000, x2 = 2.0000
b = -4.0000, w1 = -1.0000, w2 = 2.0000

forward: -----
a = 0.7311, y = 1.0000, a-y = -0.2689
db = 0.1966, dw1 = -0.1966, dw2 = 0.3932
backward: ----
db = -0.0529, dw1 = 0.0529, dw2 = -0.1058
b = -3.9894, w1 = -1.0106, w2 = 2.0212

forward: -----
a = 0.7433, y = 1.0000, a-y = -0.2567
db = 0.1908, dw1 = -0.1908, dw2 = 0.3816
backward: ----
db = -0.0490, dw1 = 0.0490, dw2 = -0.0979
b = -3.9796, w1 = -1.0204, w2 = 2.0407

